In [7]:
query = """
SELECT tjd,
       dtj.time,
       dtj.cid AS business_account_number,
       cdt AS created_dt,
       endbal,
       dtj.etc AS transaction_code,
       tamt AS transaction_amount,
       trn.dc as is_credit,
       cif.zsofiid as user_id,
       split_part(split_part(tso, 'RCID#', 2), '~', 1) AS external_account_number,
       split_part(split_part(dtj.tso, 'RINS#', 2), '~', 1) AS external_institution_id,
       split_part(split_part(dtj.tso, 'ACHCOID#', 2), '~', 1) AS originating_company_id,
       split_part(split_part(dtj.tso, 'ACHID#', 2), '~', 1) AS external_institution_trans_id,
       split_part(split_part(dtj.tso, 'ACHODFI#', 2), '~', 1) AS originator_dfi_id,
       auth.merchnm AS merchant_name
FROM dtj
LEFT JOIN authdtl auth ON auth.authid = split_part(split_part(dtj.tso, 'AUTHID#', 2), '~', 1) AND dtj.tso LIKE '%%AUTHID#%%'
JOIN profile_reporting.dep ON dep.cid = dtj.cid
JOIN cif ON cif.acn = dep.acn
LEFT JOIN trn on dtj.etc = trn.etc
WHERE tjd > '2020-09-20'
      and endbal IS NOT NULL
      and tamt IS NOT NULL
      and tamt NOT LIKE '%%#%%'
LIMIT 10
"""

In [8]:
from rdsutils.query import query_postgres

df = query_postgres(query=query,
                    port=15556,
                    database='sofi_profile_reporting')

In [9]:
df

,tjd,time,business_account_number,created_dt,endbal,transaction_code,transaction_amount,is_credit,user_id,external_account_number,external_institution_id,originating_company_id,external_institution_trans_id,originator_dfi_id,merchant_name
0,2020-09-21,00:03:05,410000155185,2020-09-21,5642.85,POSDD,47.5,1,758578,,,,,,FB *Haley Elizabeth
1,2020-09-21,00:03:05,410002565216,2020-09-21,216.55,POSDD,94,1,14563918,,,,,,Earnin-Activehours
2,2020-09-21,00:03:05,410004083314,2020-09-21,1181.83,POSDW,5.85,0,15329772,,,,,,WOODMANS FOOD M
3,2020-09-21,00:03:05,410004273950,2020-09-21,186.91,POSDW,4.34,0,15469962,,,,,,Wal-Mart Super Center
4,2020-09-21,00:03:05,410004953150,2020-09-21,198.41,POSDD,178.13,1,15917083,,,,,,"Doordash, Inc. FastPay"
5,2020-09-21,00:03:05,450002168898,2020-09-21,1421.35,DWTRF,500,0,3675597,,,,,,None
6,2020-09-21,00:03:05,450002168898,2020-09-21,1921.35,DDTRF,500,1,3675597,,,,,,None
7,2020-09-21,00:03:05,450002168898,2020-09-21,1896.35,DWTRF,25,0,3675597,,,,,,None
8,2020-09-21,00:03:05,450002168898,2020-09-21,1921.35,DDTRF,25,1,3675597,,,,,,None
9,2020-09-21,00:03:05,450002168898,2020-09-21,1896.35,DWTRF,25,0,3675597,,,,,,None


In [10]:
import pandas as pd

In [11]:
df = pd.read_feather('../../data-transactional/raw/raw_1600650783/transactions_1600650783.feather')

In [13]:
df.columns

Index(['borrower_id', 'banking_account_id', 'banking_transaction_details_id',
       'business_account_number', 'transaction_created_date_id', 'time_of_day',
       'transaction_code', 'transaction_comment', 'transaction_amount',
       'account_ending_balance', 'card_present_ind',
       'external_account_number'],
      dtype='object')

In [15]:
df.transaction_created_date_id.max()

20200920

In [16]:
df_ = pd.read_feather('../../data-transactional/labeled/labeled_1600730035.feather')

In [20]:
df_.transaction_datetime.max()

Timestamp('2020-08-30 23:59:49')

In [21]:
df_.head()

,index,transaction_datetime,borrower_id,business_account_number,transaction_code,transaction_comment,is_return,transaction_amount,account_ending_balance,card_present_ind,...,days_to_acc_close,account_closed_by_risk_in_next_90d,last_unrestricted_date_in_next_90d,latest_acc_bal,is_chg_wrt_off_in_90d,nr_returns_in_next_90d,bal_after_90d,target,target_with_restriction,indeterminate
0,0,2017-12-13 16:26:35,37722,410000007044,ACHDD,CO,False,500.00,500.00,False,...,NaN,False,False,13261.09,False,1.0,2453.03,False,False,False
1,1,2017-12-19 23:55:21,37722,410000007044,ACHDWIN,PAYPAL VERIFYBANK,False,-0.12,500.00,False,...,NaN,False,False,13261.09,False,1.0,2363.09,False,False,False
2,2,2017-12-19 23:55:21,37722,410000007044,ACHDDIN,PAYPAL VERIFYBANK,False,0.10,500.14,False,...,NaN,False,False,13261.09,False,1.0,2363.09,False,False,False
3,3,2017-12-19 23:55:21,37722,410000007044,ACHDDIN,PAYPAL VERIFYBANK,False,0.10,500.24,False,...,NaN,False,False,13261.09,False,1.0,2363.09,False,False,False
4,4,2017-12-19 23:55:21,37722,410000007044,ACHDWIN,PAYPAL VERIFYBANK,False,-0.14,500.12,False,...,NaN,False,False,13261.09,False,1.0,2363.09,False,False,False


In [ ]:
df = pd.read_feather('../../data-transactional/processed/.feather')